In [1]:
#!pip install feedparser
#!pip install newspaper3k

import feedparser as fp
import numpy as np
import json
import newspaper
from newspaper import Article
from time import mktime
from datetime import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import csv

In [2]:
# Set the limit for number of articles to download
LIMIT = 10000
articles_array = []

data = {}
data['newspapers'] = {}

# Loads the JSON files with news sites
with open('NewsPapers.json') as data_file:
    companies = json.load(data_file)

In [3]:
count = 1

# Iterate through each news company
for company, value in companies.items():
    # If a RSS link is provided in the JSON file, this will be the first choice.
    # Reason for this is that, RSS feeds often give more consistent and correct data. RSS (Rich Site Summary; originally RDF Site Summary; often called Really Simple Syndication) is a type of
    # web feed which allows users to access updates to online content in a standardized, computer-readable format
    # If you do not want to scrape from the RSS-feed, just leave the RSS attr empty in the JSON file.
    if 'rss' in value:
        d = fp.parse(value['rss'])
        print("Downloading articles from ", company)
        newsPaper = {
            "rss": value['rss'],
            "link": value['link'],
            "articles": []
        }
        for entry in d.entries:
            # Check if publish date is provided, if no the article is skipped.
            # This is done to keep consistency in the data and to keep the script from crashing.
            if hasattr(entry, 'published'):
                if count > LIMIT:
                    break
                article = {}
                article['link'] = entry.link
                date = entry.published_parsed
                article['published'] = datetime.fromtimestamp(mktime(date)).isoformat()
                try:
                    content = Article(entry.link)
                    content.download()
                    content.parse()
                except Exception as e:
                    # If the download for some reason fails (ex. 404) the script will continue downloading
                    # the next article.
                    print(e)
                    print("continuing...")
                    continue
                article['title'] = content.title
                article['text'] = content.text
                article['authors'] = content.authors
                newsPaper['articles'].append(article)
                articles_array.append(article)
                print(count, "articles downloaded from", company, ", url: ", entry.link)
                count = count + 1
    else:
        # This is the fallback method if a RSS-feed link is not provided.
        # It uses the python newspaper library to extract articles
        print("Building site for ", company)
        paper = newspaper.build(value['link'], memoize_articles=False)
        newsPaper = {
            "link": value['link'],
            "articles": []
        }
        noneTypeCount = 0
        for content in paper.articles:
            if count > LIMIT:
                break
            try:
                content.download()
                content.parse()
            except Exception as e:
                print(e)
                print("continuing...")
                continue
            # Again, for consistency, if there is no found publish date the article will be skipped.
            # After 10 downloaded articles from the same newspaper without publish date, the company will be skipped.

            article = {}
            article['title'] = content.title
            article['authors'] = content.authors
            article['text'] = content.text
            article['link'] = content.url
            article['published'] = content.publish_date
            newsPaper['articles'].append(article)
            articles_array.append(article)
            print(count, "articles downloaded from", company, " using newspaper, url: ", content.url)
            count = count + 1
            #noneTypeCount = 0
    count = 1
    data['newspapers'][company] = newsPaper


Building site for  cnn
1 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/business/media
2 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2017/03/04/vr/how-to-watch-vr
3 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/12/04/uk/brexit-uk-eu-legal-position-gbr-intl/index.html
4 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/12/04/uk/brexit-legal-advice-contempt-of-court-gbr-intl/index.html
5 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/12/04/tech/digital-sales-tax-tech-europe/index.html
6 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/12/04/tech/london-blue-email-hackers/index.html
7 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/10/15/uk/meghan-harry-pregnant-intl/index.html
8 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/12/04/health/london-skeleton-thames-boo

54 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/12/03/perspectives/george-h-w-bush-plane-ride/index.html
55 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/12/04/success/microsoft-accessibility-jenny-lay-flurrie/index.html
56 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/12/04/tech/blade-helicopters-india-expansion/index.html
57 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/11/29/success/red-hat-ceo-jim-whitehurst/index.html
58 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/12/04/success/office-holiday-party-advance-career/index.html
59 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/12/04/cars/porsche-911/index.html
60 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/12/04/success/only-ones-diversity/index.html
61 articles downloaded from cnn  using newspaper, url:  http://www.c

115 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/08/23/us/cnnheroes-florence-phillips-esl-in-home-program-of-northern-nevada/index.html
116 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/08/16/us/cnnheroes-ellen-stackable-poetic-justice/index.html
117 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/05/17/us/cnnheroes-paul-steklenski-flying-fur-animal-rescue/index.html
118 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/07/19/health/cnnheroes-maria-rose-belding-means/index.html
119 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/06/28/us/cnnheroes-luke-mickelson-sleep-in-heavenly-peace/index.html
120 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2018/04/12/health/cnnheroes-amanda-boxtel-bridging-bionics/index.html
121 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/interactive/2017/02/health

175 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2016/09/19/politics/trump-supporters-working-class-white-kaiser-family-foundation-infographic/index.html
176 articles downloaded from cnn  using newspaper, url:  http://money.cnn.com/2016/09/23/news/economy/white-working-class-economy/index.html
177 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2017/08/15/world/iyw-aid-sierra-leone-mudslide-victims/index.html
178 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2017/03/16/world/iyw-how-to-help-east-africa/index.html
179 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2016/10/19/middleeast/iyw-mosul-iraq-how-to-help/index.html
180 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2016/05/10/health/iyw-prescription-drug-abuse-how-to-help-health/index.html
181 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/2017/08/07/world/iyw-impact-of-youssif/i

238 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/style/article/honjok-south-korea-loner-culture/index.html
239 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/style/article/underwater-photography-style/index.html
240 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/style/article/susan-wood-portraits/index.html
241 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/style/article/icon-derelict-vintage-cars/index.html
242 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/style/article/moon-ribas-cyborg-smart-creativity/index.html
243 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/style/article/wwii-bunkers-d-day-photos/index.html
244 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/style/article/zacharie-gaudrillot-roy-facades/index.html
245 articles downloaded from cnn  using newspaper, url:  http://www.cnn.com/style/article/

...

591 articles downloaded from bloomberg  using newspaper, url:  https://www.bloomberg.com/news/articles/2018-10-22/alberta-premier-urges-federal-intervention-on-crude-by-rail
592 articles downloaded from bloomberg  using newspaper, url:  https://www.bloomberg.com/news/articles/2018-11-14/ams-is-latest-apple-supplier-to-cut-sales-estimates-within-days
593 articles downloaded from bloomberg  using newspaper, url:  https://www.bloomberg.com/opinion/articles/2017-12-20/coal-s-resilience-will-be-its-downfall
594 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/opinion/corporate-governance-theres-no-such-thing-as-a-part-time-watchdog
595 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/opinion/board-failures-five-questions-to-ask-on-why-boards-are-ineffective
596 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/opinion/board-failures-the-travails-of-corporate-governance-w

639 articles downloaded from bloomberg  using newspaper, url:  https://www.washingtonpost.com/news/theworldpost/wp/2018/03/06/xi/?utm_term=.b3ca00ba6a25
640 articles downloaded from bloomberg  using newspaper, url:  https://www.bloomberg.com/news/articles/2018-12-04/toll-brothers-home-orders-plunge-led-by-slowdown-in-california
641 articles downloaded from bloomberg  using newspaper, url:  https://www.bloomberg.com/news/articles/2018-10-18/more-bad-news-for-builders-with-bofa-downgrades-rbc-pessimism
642 articles downloaded from bloomberg  using newspaper, url:  https://www.bloomberg.com/politics/articles/2018-03-05/a-1-39-billion-hydro-deal-in-limbo-as-peru-debates-fine-on-sale
643 articles downloaded from bloomberg  using newspaper, url:  https://www.bloomberg.com/news/articles/2018-12-03/altria-growth-quest-intensifies-with-report-of-cronos-weed-talks
644 articles downloaded from bloomberg  using newspaper, url:  https://www.bloomberg.com/news/articles/2018-11-28/juul-stake-would-le

687 articles downloaded from bloomberg  using newspaper, url:  http://www.bloombergquint.com/politics/u-k-government-cranks-up-no-deal-planning-brexit-update
688 articles downloaded from bloomberg  using newspaper, url:  http://www.bloombergquint.com/markets/oil-gains-as-saudi-russia-pact-and-canada-cuts-ease-supply-fears
689 articles downloaded from bloomberg  using newspaper, url:  http://www.bloombergquint.com/business/ilfs-to-slash-65-jobs-to-rescue-group
690 articles downloaded from bloomberg  using newspaper, url:  http://www.bloombergquint.com/business/axis-capital-names-co-ceos-as-dharmesh-mehta-resigns
691 articles downloaded from bloomberg  using newspaper, url:  http://www.bloombergquint.com/business/kesoram-industries-to-hive-off-tyre-business-list-new-entity
692 articles downloaded from bloomberg  using newspaper, url:  http://www.bloombergquint.com/business/india-to-allow-business-visa-extension-for-up-to-15-years
693 articles downloaded from bloomberg  using newspaper, u

737 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/view/excessive-state-fines-how-much-is-too-much
738 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/view/trump-s-wall-still-isn-t-funded-thanks-to-republicans
739 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/view/maybe-we-have-the-economic-growth-equation-backward
740 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/view/u-s-job-market-gives-workers-what-they-want-stability
741 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/view/netanyahu-s-case-4000-scandal-sounds-a-little-like-trump
742 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/view/syria-s-assad-has-gassed-his-own-people-again
743 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/view/e

788 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/portfolio/bqportfolio-investment-thumb-rules-for-all-generations-this-diwali
789 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/bqportfolio-videos/bqportfolio-can-harsha-vardhan-build-a-house-costing-rs-5-crore-by-2028
790 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/portfolio/bqportfolio-will-gautam-be-able-to-save-rs-6-crore-to-retire-by-2048
791 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/bqportfolio-videos/bqportfolio-three-different-saving-plans-for-your-childs-education
792 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/portfolio/bqportfolio-in-which-sagar-gets-his-holiday-and-a-solid-start-to-portfolio-building-2
793 articles downloaded from bloomberg  using newspaper, url:  https://www.bloombergquint.com/portfol

In [4]:
#Finally it saves the articles as a CSV-file.
try:
    f = csv.writer(open('Scraped_data_news_output.csv', 'w', encoding='utf-8'))
    f.writerow(['Title', 'Authors','Text','Link','Published_Date'])
    #print(article)
    for artist_name in articles_array:
        title = artist_name['title']
        authors=artist_name['authors']
        text=artist_name['text']
        link=artist_name['link']
        publish_date=artist_name['published']
        # Add each artist’s name and associated link to a row
        f.writerow([title, authors, text, link, publish_date])
except Exception as e: print(e)